In [ ]:
Practical 9 - Implement object detection from the image

!pip install tensorflow
!pip install opencv-python
!pip install numpy


import cv2
import numpy as np
import matplotlib.pyplot as plt


#Load YOLO
net = cv2.dnn.readNet("dataset/yolov3.cfg", "dataset/yolov3.weights") # Original yolov3
#net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg") #Tiny Yolo
classes = []
with open("dataset/coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

# layer_names = net.getLayerNames()
# outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


layer_names = net.getLayerNames()
print("Layer Names:", layer_names)  # Print layer names for debugging

outputlayers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
print("Output Layers:", outputlayers)  # Print output layers for debugging

colors= np.random.uniform(0,255,size=(len(classes),3))




# Load an image
img = cv2.imread("PANDA.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(outputlayers)





class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-max Suppression
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Show the bounding boxes
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = (0, 255, 0)
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

# Display image using matplotlib
plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()